### Objectives

(1) Minimize the number of station sides activated and penalize their activation using weights μ1 and μ2.

(2) Minimize the total cost of using agents C(a) (humans or robots).

### Contraints

(3) Each task j must be assigned to exactly one mated-station m and one side h.

(4) Tasks that are performed on beneath-side stations (set us) must be assigned to one station from the set MU (mated stations with beneath-side infrastructure).

(5) Tasks that are performed on above-side stations (set ps) must be assigned to one station from the set MP (mated stations with above-side infrastructure).

(6) An agent a can only be assigned to one station and side at a time.

(7) Humans are only assigned to the established station: A side h in station m can only be activated if an agent a is assigned to it.

(8) No agents from the subset o(a) are assigned to side 4 (above-side) in any station m.
o(a): Refers to a subset of agents, presumably humans, as tasks on the above-side (side 4) are meant to be performed exclusively by robots.

**Constraint (9) and (10) define the cycle time which each task must be finished during the cycle time:**

(9) The finishing time of task j for product model n must not exceed the cycle time cy.

(10) The finishing time of task j for product model n must be greater than or equal to its real processing time r.

(11) Calculates the real processing time for each task j and model n, accounting for the agent performing it, the station, and the side.

**Constraints (12) and (13) are for tasks that include in pi(j) (Set of immediate predecessors of task j)**

(12) 2 tasks have different mated-station: If task i is a predecessor of task j, task i must be assigned to a station earlier (or the same station as) task j.

(13) 2 tasks are in the same mated-station: Task j starts only after its predecessor task i is completed. The ϕ-based terms deactivate this constraint when the tasks are not assigned to the same station-side configuration.

**Constraints (14) and (15) are for tasks which do not have any predecessor and successor relationships.**

(14) The precedence relationship and timing between tasks j and k are respected, but the assignment of k to its station-side xkmh is directly required.

(15) Timing relationship is reversed, i.e., task k must complete before task j starts, based on the sequence variable yjk, while maintaining strict enforcement on xkmh.

**Constraints (16) and (17) calculate the number of different mated-stations which include one-sided mated-stations, two-sided mated-stations, three-sided mated-stations and four-sided mated-stations.**

(16) Tasks j can only be assigned to a station-side if that side zmh is activated. (If zmh=0, this constraint ensures no tasks are assigned to that side).

(17) Ensures consistency between the number of sides zmh utilized in a station m and the values of α,β,θ,γ (If zmh indicates one side is used, α=1, and β,θ,γ=0).

**Constraints (18) and (19) define zoning constraints that include positive and negative constraint.**

(18) More than one task is assigned to a station: Tasks j and i, which have a positive constraint (po), are assigned to the same side h.

(19) Only one task is assigned to a station: Tasks j and i, which have a negative constraint (ne), are not assigned to the same side of station m.

**Constraints (20) and (21) define the synchronous constraint which means if two tasks must be done together synchronously, they would be assigned to the same mated-station.**

(20) Tasks j and i, which have a synchronous constraint (se), are assigned to different sides l≠h of the same station m.

(21) Tasks j and i, which have a synchronous constraint (se), have synchronized finishing times across sides.

**In constraints (22) and (24), if the task must be performed by a human, ensure that human is assigned to that station.** 

(22) vI,mh is activated if and only if at least one task in to (Set of tasks which humans perform) is assigned to station m, side h.

(24) Prevents agents from the subset r(a) from being assigned to sides where tasks from to are assigned.

**In constraints (23) and (25),if the task must be performed by a robot, ensure that the robot is assigned to that station.**

(23) vII,mh is activated only if tasks from tr are assigned.

(25) Prevents agents from the subset o(a) from being assigned to sides where tasks from tr are assigned.

In [ ]:
from gurobipy import Model, GRB, quicksum
import random
import pandas as pd
from collections import defaultdict

# Create model
model = Model("4-Sided ALB")

In [ ]:
# sets
tasks = []  # Set of tasks
mated_stations = []  # Set of mated stations
sides = [1, 2, 3, 4] # Sides (1=left, 2=right, 3=beneath, 4=above)
agents = []  # Set of agents
product_models = []  # Set of product models

In [ ]:
J = len(tasks)
M = len(mated_stations)
N = len(product_models)
H = len(sides)
A = len(agents)

In [ ]:
# Parameters
# Mated-stations
MU = set()  # with beneath-side tasks
MP = set()  # with above-side tasks

# Subsets of tasks: Tasks accomplished at a X-side station
ls = set()  # left
rs = set()  # right 
bs = set()  # both left and right
us = set()  # beneath
ps = set()  # above

# Tasks based on agents
to = set()  # Tasks humans perform
tr = set()  # Tasks robots perform
tb = set()  # Tasks both humans and robots can perform

# Task dependencies
sa = {j: set() for j in tasks}  # All successors of task j
si = {j: set() for j in tasks}  # Immediate successors of task j
pa = {j: set() for j in tasks}  # All predecessors of task j
pi = {j: set() for j in tasks}  # Immediate predecessors of task j
pn = set()  # Tasks with no immediate predecessors

# Costs and times
C = {a: cost_a for a in agents}  # Cost of utilizing agent `a`
cy = 0  # Cycle time (define a value or decision variable)
t_ajn = {(j, n, a): time_jna for j in tasks for n in product_models for a in agents}  # Processing time of task `j` for model `n` by agent `a`

# Mated-stations
f = {(m, h): mated_station_set for m in mated_stations for h in sides}  # Set of mated-stations with assignment side `h`

# Zoning constraints: Pairs of tasks for X zoning constraints
po = set()  # positive 
ne = set()  # negative
se = set()  # synchronous constraints

# Weights and constraints
phi = 1e6  # Large number
mu_1 = 0.6  # Opened mated-station weight (define a value)
mu_2 = 0.3  # Opened station weight (define a value)

### Numeric example

In [ ]:
A = 20  # Number of agents
J = 30  # Number of tasks
M = 5   # Number of mated stations
N = 3   # Number of product models
H = [1, 2, 3, 4]  # Sides

In [ ]:
# Provided data (data_modified)
robots = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}  
humans = {10, 11, 12, 13, 14, 15, 16, 17, 18, 19}  
ls = [0, 1, 2, 3]  
bs = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]  
rs = [15, 16, 17, 18, 19] 
us = [20, 21, 22, 23, 24, 25]  
ps = [26, 27, 28, 29]  
tb = [0, 2, 3, 5, 7, 9, 11, 13, 14, 16, 18]  
to = [1, 4, 6, 12, 17]  
tr = [8, 10, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]  
MU = [1, 2]  
MP = [2, 3]  
f_mh = [(0, 0), (0, 1), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (2, 3),
        (3, 0), (3, 1), (3, 3), (4, 0), (4, 1)] 
cy = 100
phi = 999000000
mu_1 = 0.6
mu_2 = 0.3
C = {a: random.uniform(30, 50) for a in range(A)}
t_ajn = {(j, n, a): random.uniform(3, 15) for j in range(J) for a in range(A) for n in range(N)}

po = {(0, 1)}  
ne = {(0, 29), (0, 4)}  
se = {(3, 5)}  
sa = {
    (1, 7), (1, 8), (3, 10), (4, 9), (4, 29), (8, 11), (8, 13), (10, 11),
    (10, 15), (14, 22), (14, 29), (1, 11), (1, 13), (3, 11), (3, 15)
}
pa = {
    (7, 1), (8, 1), (10, 3), (9, 4), (29, 4), (11, 8), (13, 8), (11, 10), (15, 10),
    (22, 14), (29, 14), (11, 1), (13, 1), (11, 3), (15, 3)
}

# Define robots and humans sub-sets
def r(agents=list(range(A))):
    return [a for a in agents if a in robots]

def o(agents=list(range(A))):
    return [a for a in agents if a in humans] 

In [ ]:
# Initialize pi with empty sets for all tasks
pi = {j: set() for j in range(J)}

# Using pa
for (j, i) in pa:  # (j,i) means j is a predecessor of i
    if (j, i) not in sa:  # If j is not an indirect predecessor via another task
        pi[i].add(j)

# Using sa
for (i, j) in sa:  # (i,j) means i is a direct predecessor of j
    if (j, i) not in pa:  # If j is not an indirect successor via another task
        pi[j].add(i)

# Include se relationships in pi
for (i, j) in se:
    pi[j].add(i)
    pi[i].add(j) 

In [ ]:
# Convert sa,pa to dictionaries
sa_dict = defaultdict(set)
for key, value in sa:
    sa_dict[key].add(value)

pa_dict = defaultdict(set)
for key, value in pa:
    pa_dict[key].add(value)

sa = dict(sa_dict)
pa = dict(pa_dict)

for j in range(J):
    sa.setdefault(j, set())  # Add task with empty successors if missing
    pa.setdefault(j, set()) 

In [ ]:
# Task sets
ls = set(ls)  # Left-side tasks
rs = set(rs)  # Right-side tasks
bs = set(bs)  # Beneath-side tasks
us = set(us)  # Beneath-side tasks
ps = set(ps)  # Above-side tasks

# Tasks with opposite directions
D_j = {}
for j in range(J):
    if j in rs:
        D_j[j] = ls.union(us).union(ps)
    elif j in ls:
        D_j[j] = rs.union(us).union(ps)
    elif j in bs:
        D_j[j] = us.union(ps)
    elif j in us:
        D_j[j] = ls.union(rs).union(bs).union(ps)
    elif j in ps:
        D_j[j] = ls.union(rs).union(us).union(bs)
    else:
        D_j[j] = set()  # No opposite tasks
        
# Preferred sides (directions) for task j
H_j = {}
for j in range(J):
    if j in rs:
        H_j[j] = {1}  # Right-side
    elif j in ls:
        H_j[j] = {2}  # Left-side
    elif j in us:
        H_j[j] = {3}  # Beneath-side
    elif j in ps:
        H_j[j] = {4}  # Above-side
    elif j in bs:
        H_j[j] = {1, 2}  # Left and right sides
    else:
        H_j[j] = set()  # No valid sides (optional)

In [ ]:
# Variables
x = model.addVars(J, M, H, vtype=GRB.BINARY, name="x")  # Task assignment to mated-station and side
z = model.addVars(M, H, vtype=GRB.BINARY, name="z")  # Station-side utilization
y = model.addVars(J, J, vtype=GRB.BINARY, name="y") #task j is assigned earlier than task k in the same station
q = model.addVars(A, H, M, vtype=GRB.BINARY, name="q")  # Agent assignment to station-side
t_f = model.addVars(J, N, vtype=GRB.CONTINUOUS, name="t_f")  # Finishing time
rr = model.addVars(J, N, vtype=GRB.CONTINUOUS, name="rr")  # Real time
alpha = model.addVars(M, vtype=GRB.BINARY, name="alpha")
beta = model.addVars(M, vtype=GRB.BINARY, name="beta")
theta = model.addVars(M, vtype=GRB.BINARY, name="theta")
gamma = model.addVars(M, vtype=GRB.BINARY, name="gamma")
v_I = model.addVars(M, H, vtype=GRB.BINARY, name="v_I")
v_II = model.addVars(M, H, vtype=GRB.BINARY, name="v_II")
cy = model.addVar(lb=0, name="cy")

In [ ]:
# Objective (1): Minimize number of stations and mated-stations
model.setObjective(
    mu_1 * quicksum(alpha[m] + beta[m] + theta[m] + gamma[m] for m in range(M)) +
    mu_2 * quicksum(z[m, h] for m in range(M) for h in sides),
    GRB.MINIMIZE
)

# Objective (2): Minimize agent utilization cost
#agent_cost = quicksum(C[a] * quicksum(q[a, h, m] for h in sides for m in range(M)) for a in range(A))
#model.setObjective(agent_cost, GRB.MINIMIZE)

In [ ]:
# Constraint (3): Each task is assigned to only one side of a mated-station
model.addConstrs(
    (quicksum(x[j, m, h] for m in range(M) for h in sides) == 1 for j in range(J)), 
    name="task_assignment"
)

In [ ]:
# Constraint (4): Beneath-side tasks are assigned to beneath stations
model.addConstrs(
    (quicksum(x[j, m, h] for m in MU for h in sides) == 1 for j in us),
    name="beneath_side_task_assignment"
)

In [ ]:
# Constraint (5): Above-side tasks are assigned to above stations
model.addConstrs(
    (quicksum(x[j, m, h] for m in MP for h in sides if h in [4]) == 1 for j in ps if j < J), # Ensure tasks are within range
    name="above_side_task_assignment"
)

In [ ]:
#

In [ ]:
# Constraint (6): Each agent is assigned to at most one station
model.addConstrs(
    (quicksum(q[a, h, m] for m in range(M) for h in sides) <= 1 for a in range(A)),
    name="agent_assignment"
)

In [ ]:
# Constraint (7): Humans are only assigned to established stations 
# range(A) or  o(agents)
model.addConstrs(
    (quicksum(q[a, h, m] for a in range(A)) == z[m, h] for m in range(M) for h in sides),
    name="human_station_assignment"
)

In [ ]:
# Constraint (8): Above-side tasks are done only by robots
model.addConstrs(
    (quicksum(q[a, 4, m] for a in o(agents)) == 0 for m in range(M)),
    name="robots_only_above"
)

In [ ]:
# Constraint (9): Task finish time ≤ cycle time
model.addConstrs(
    (t_f[j, n] <= cy for j in range(J) for n in range(N)),
    name="cycle_time_upper_bound"
)

In [ ]:
# Constraint (10): Task finish time ≥ real task time
model.addConstrs(
    (t_f[j, n] >= rr[j, n] for j in range(J) for n in range(N)),
    name="cycle_time_lower_bound"
)

In [ ]:
# Constraint (11): rr_jn calculation
model.addConstrs(
    (
        rr[j, n] == quicksum(
            t_ajn[j, n, a] * x[j, m, h] * q[a, h, m]
            for h in sides
            for a in range(A)
            for m in range(M)
        )
        for j in range(J)
        for n in range(N)
    ),
    name="real_time_calculation"
)

In [ ]:
# Constraint (12): Task precedence
model.addConstrs(
    (
        quicksum(s * x[i, s, h] for s in range(M) for h in sides) <=
        quicksum(m * x[j, m, h] for m in range(M) for h in sides)
        for j in range(J) if j not in pn
        for i in pi[j]  # Immediate predecessors of task j
    ),
    name="task_precedence"
)

In [ ]:
# Constraint (13): Task timing
phi = 1e6  # Large constant
model.addConstrs(
    (
        t_f[j, n] - t_f[i, n]
        + phi * (1 - quicksum(x[i, m, h] for h in sides))
        + phi * (1 - quicksum(x[j, m, h] for h in sides))
        >= rr[j, n]
        for j in range(J) if j not in pn
        for i in pi[j]  # Immediate predecessors of task j
        for m in range(M)
        for n in range(N)
    ),
    name="task_timing"
)

In [ ]:
# Constraint (14): Task timing with precedence
model.addConstrs(
    (
        t_f[k, n] - t_f[j, n] 
        + phi * (1 - x[j, m, h]) 
        + (1 - x[k, m, h]) 
        + phi * (1 - y[j, k]) 
        >= rr[k, n]
        for j in range(J)
        for n in range(N)
        for k in range(J) if k != j and k not in pa[j] | sa[j] | D_j[j]
        for m in range(M)
        for h in H_j[j] & H_j[k]
    ),
    name="task_timing_precedence"
)

In [ ]:
# Constraint (15): Reverse task timing
model.addConstrs(
    (
        t_f[j, n] - t_f[k, n] 
        + phi * (1 - x[j, m, h])  
        + (1 - x[k, m, h])  
        + phi * y[j, k]           
        >= rr[j, n]
        for j in range(J)
        for n in range(N)
        for k in range(J) if k != j and k not in pa[j] | sa[j] | D_j[j]
        for m in range(M)
        for h in H_j[j] & H_j[k]
    ),
    name="reverse_task_timing"
)

In [ ]:
# Constraint (16): Task assignment and station utilization
model.addConstrs(
    (quicksum(x[j, m, h] for j in range(J)) - phi * z[m, h] <= 0 for m in range(M) for h in sides),
    name="station_task_assignment"
)

In [ ]:
# Constraint (17): Station utilization logic
model.addConstrs(
    (quicksum(z[m, h] for h in sides) - 4 * gamma[m] - 3 * beta[m] - 2 * alpha[m] == 0 for m in range(M)),
    name="station_utilization"
)

In [ ]:
# Constraint (18): Opposite task assignment
model.addConstrs(
    (x[j, m, h] - x[i, m, h] == 0 for (j, i) in po for m in range(M) for h in H_j[j] & H_j[i]),
    name="opposite_task_assignment"
)

In [ ]:
# Constraint (19): Task assignment limits by sides
model.addConstrs(
    (quicksum(x[j, m, h] for h in H_j[j]) + quicksum(x[i, m, h] for h in H_j[i]) <= 1
     for (j, i) in ne for m in range(M) if set(H_j[j]).intersection(H_j[i])),
    name="task_assignment_limit"
)

In [ ]:
# Constraint (20): Distinct station assignments
model.addConstrs(
    (
        x[j, m, l] - x[i, m, h] == 0
        for (j, i) in se
        for m in range(M)
        for h in H_j[i]
        for l in H_j[j]
        if h != l
    ),
    name="distinct_station_assignment"
)

In [ ]:
# Constraint (21): Synchronize task times
model.addConstrs(
    (t_f[j, n] - rr[j, n] == t_f[i, n] - rr[i, n] for (j, i) in se for n in range(N)),
    name="task_time_sync"
)

In [ ]:
# Constraint (22): Station capacity for vI
model.addConstrs(
    (quicksum(x[j, m, h] for j in to) <= phi * v_I[m, h] for m in range(M) for h in H_j[j]),
    name="station_capacity_vI"
)

In [ ]:
# Constraint (23): Station capacity for vII
model.addConstrs(
    (quicksum(x[j, m, h] for j in tr) <= phi * v_II[m, h] for m in range(M) for h in H_j[j]),
    name="station_capacity_vII"
)

In [ ]:
# Constraint (24): Agent assignment limits for vI
model.addConstrs(
    (quicksum(q[a, m, h] for a in r(agents)) <= 1 - v_I[m, h] for m in range(M) for h in sides),
    name="agent_assignment_limit_vI"
)

In [ ]:
# Constraint (25): Agent assignment limits for vII
model.addConstrs(
    (quicksum(q[a, m, h] for a in r(agents)) <= 1 - v_II[m, h] for m in range(M) for h in sides),
    name="agent_assignment_limit_vII"
)

In [ ]:
# Possible Extension 

In [ ]:
model.update()
print("Number of variables:", model.NumVars)
print("Number of constraints:", model.NumConstrs)

In [ ]:
# Optimize the model
model.optimize()

In [ ]:
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
    
    # Objective value
    print(f"Objective Value: {model.ObjVal}")
    
    # Initialize list to store DataFrame rows
    result_data = []
    
    for m in range(M):  # Loop over stations
        for h in sides:  # Loop over sides
            tasks_assigned = []
            agents_assigned = []
            completion_times = {"A": 0, "B": 0, "C": 0}
            
            for j in range(J):  # Loop over tasks
                if x[j, m, h].X > 0.5:  
                    tasks_assigned.append(j)
                    for n in range(N):  
                        completion_times[chr(65 + n)] += t_ajn[j, n, 0] # Completion time for each product
            
            for a in range(A):  # Loop over agents
                if q[a, h, m].X > 0.5:  
                    agents_assigned.append(a)
            
            if tasks_assigned:  
                result_data.append({
                    "Station": m + 1,
                    "Tasks": ", ".join(map(str, tasks_assigned)),
                    "Side": "left" if h == 1 else "right" if h == 2 else "beneath" if h == 3 else "above",
                    "Agent Number": ", ".join(map(str, agents_assigned)),
                    "Completion Time A": completion_times["A"],
                    "Completion Time B": completion_times["B"],
                    "Completion Time C": completion_times["C"]
                })
    
    # Convert to DataFrame
    result_df = pd.DataFrame(result_data)
    
    # Display the DataFrame
    print(result_df)
else:
    # Handle errors
    if model.Status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.Status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.Status}.")

In [ ]:
# Check the optimization status
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
    
    # Objective value
    print(f"Objective Value: {model.ObjVal}")
    
    # Values of decision variables
    print("\nDecision Variables:")
    for v in model.getVars():
        if v.X > 1e-6:  
            print(f"{v.VarName}: {v.X}")
    
    # Task Assignments
    print("\nTask Assignments:")
    for j in range(J):
        for m in range(M):
            for h in sides:
                if x[j, m, h].X > 0.5:  
                    print(f"Task {j} is assigned to mated station {m} on side {h}.")
                    
    print("\nAgent Assignments:")
    for a in range(A):
        for m in range(M):
            for h in sides:
                if q[a, h, m].X > 0.5:  
                    print(f"Agent {a} is assigned to mated station {m} on side {h}.")
else:
    # Handle errors
    if model.Status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.Status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.Status}.")

In [ ]:
# Debugging
#model.computeIIS()
#model.write("model.ilp")

In [ ]:
#for c in model.getConstrs():
    #if c.IISConstr:
        #print(f"Infeasible Constraint: {c.constrName}")

#for v in model.getVars():
    #if v.IISLB:
        #print(f"Infeasible Lower Bound on Variable: {v.VarName}")
    #if v.IISUB:
        #print(f"Infeasible Upper Bound on Variable: {v.VarName}")